
## Demonstrating Training of DDPM (On original images not on latent space)

Objective: Train a Denoising Diffusion Probabilistic Model (DDPM) on the
smithsonian-butterflies-subset at 128x128 resolution, using a U-Net backbone.

What this file contains (cell-like sections):
  1) imports & helpers
  2) configuration
  3) dataset loading & transforms
  4) U-Net model definition
  5) DDPM pipeline (noise schedule, q_sample and p_sample steps)
  6) training loop + checkpointing
  7) sampling (generate ~50 images) and plotting

Notes:
- This is a complete, runnable script designed to be used in a Jupyter cell
  environment. Cells are delimited by `# %%` so you can paste into a .py file
  and use in VSCode/Jupyter or split into notebook cells.
- Replace `DATA_DIR` with the path where you've downloaded the
  smithsonian-butterflies-subset image folder. The script includes an optional
  Hugging Face datasets download snippet (requires `datasets` & internet).
- Training hyperparameters are conservative to be runnable on a single GPU.
  Adjust batch_size, epochs, and learning rate to your compute.

References
- Ho et al., 2020: Denoising Diffusion Probabilistic Models
- Implementation inspiration: mattroz/diffusion-ddpm (GitHub)

# Section 1 — Imports & What They Do

This section loads all the libraries needed for training a DDPM model.

---

## Core Python Utilities
Used for file handling, randomness, and working with paths:

- os  - file and directory operations  
- glob - find files (e.g., *.jpg, *.png)  
- random - random number utilities  
- pathlib.Path - clean path handling  
- datetime - timestamps for logging  

---

## Numerical Tools

- numpy - fast numerical operations  
- tqdm.auto - clean progress bars in Jupyter  
- math - basic math functions  

---

## PyTorch (Deep Learning Framework)

- torch - tensors and GPU support  
- torch.nn, torch.nn.functional - neural network layers and activations  
- Dataset, DataLoader - data batching pipeline  

---

## Image Processing

- torchvision.transforms - resizing, cropping, normalization  
- PIL.Image - load JPEG/PNG images  

---

## Visualization

- matplotlib.pyplot - plot loss curves and sample outputs  

---

In summary: these imports set up everything needed for loading data, training the U-Net, running diffusion steps, and visualizing results.

In [1]:
# 1) Imports & helpers
import os
import math
import glob
import random
from pathlib import Path
from datetime import datetime


import numpy as np
from tqdm.auto import tqdm


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


#import matplotlib.pyplot as plt

In [2]:
# set deterministic seeds for reproducibility (optional)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
# 2) Configuration
CONFIG = {
# dataset
"data_dir": "./smithsonian-butterflies-subset/images", # change to your path
"image_size": 128,
"channels": 3,


# training
"batch_size": 8,
"epochs": 200, # set 50 for quicker runs; increase to 300 for final
"lr": 2e-4,
"num_workers": 4,


# diffusion
"timesteps": 1000,
"beta_start": 1e-4,
"beta_end": 0.02,


# logging & checkpoint
"save_dir": "./ddpm_checkpoints",
"save_every": 5, # epochs
"device": "cuda" if torch.cuda.is_available() else "cpu",
}


os.makedirs(CONFIG["save_dir"], exist_ok=True)
print("Using device:", CONFIG["device"])

Using device: cuda


# Dataset Loading & Transforms

This class loads all images from a folder and prepares them for training the DDPM model.

---

## 🔹 What the Dataset Class Does

### `ImageFolderDataset`
A custom PyTorch `Dataset` that:

1. **Finds all images** inside a directory  
   - Searches for `*.jpg` and `*.png` recursively  
   - Stores file paths in `self.files`  
   - Throws an error if no images are found  

2. **Applies transforms** to each image  
   Default transforms include:
   - Resize → `(128,128)`
   - Center crop  
   - Random horizontal flip (augmentation)  
   - Convert to tensor  
   - Normalize to `[-1, 1]`  

3. **Returns an image tensor** ready for training.

---

## 🔹 Key Methods

### `__len__()`
Returns the number of images in the dataset.

### `__getitem__(idx)`
- Loads an image from disk  
- Converts to RGB  
- Applies transforms  
- Returns the processed tensor  

---

This dataset class is ideal for training diffusion models because it ensures all images are consistently sized, normalized, and augmented.


In [4]:
class ImageFolderDataset(Dataset):
    def __init__(self, folder, image_size=128, channels=3, transforms_=None):
        self.files = sorted(glob.glob(os.path.join(folder, "**", "*.jpg"), recursive=True) +
        glob.glob(os.path.join(folder, "**", "*.png"), recursive=True))
        if len(self.files) == 0:
            raise RuntimeError(f"No images found in {folder}")
        self.image_size = image_size
        self.channels = channels
        if transforms_ is None:
            self.transforms = transforms.Compose([
                    
                    transforms.ToTensor(),

                    transforms.Resize((image_size, image_size)),
                    transforms.CenterCrop(image_size),

                    # --- Geometric augments ---
                    transforms.RandomHorizontalFlip(p=0.5),

                    # Small rotation (safe for diffusion)
                    transforms.RandomApply([
                        transforms.RandomRotation(degrees=10)
                    ], p=0.2),

                    # # --- Photometric augments ---
                    # transforms.RandomApply([
                    #     transforms.ColorJitter(
                    #         brightness=0.1,
                    #         contrast=0.1,
                    #         saturation=0.1,
                    #         hue=0.02
                    #     )
                    # ], p=0.3),

                    # # Mild Gaussian blur
                    # transforms.RandomApply([
                    #     transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0))
                    # ], p=0.15),

                    # # Mild Gaussian noise (custom)
                    # transforms.RandomApply([
                    #     transforms.Lambda(lambda x: x + 0.01 * torch.randn_like(x))
                    # ], p=0.2),
                    transforms.Normalize([0.5] * channels, [0.5] * channels),
                ])
            # self.transforms = transforms.Compose([
            #     transforms.Resize((image_size, image_size)),
            #     transforms.CenterCrop(image_size),
            #     transforms.RandomHorizontalFlip(),
            #     transforms.ToTensor(),
            #     transforms.Normalize([0.5]*channels, [0.5]*channels),
            # ])
        else:
            self.transforms = transforms_
    
    
    def __len__(self):
        return len(self.files)
    
    
    def __getitem__(self, idx):
        path = self.files[idx]
        img = Image.open(path).convert('RGB')
        img = self.transforms(img)
        return img

!pip install datasets


In [5]:
import sys
!{sys.executable} -m pip install datasets


import os
from datasets import load_dataset
from PIL import Image

OUT_DIR = "./smithsonian-butterflies-subset/images"
os.makedirs(OUT_DIR, exist_ok=True)

ds = load_dataset("huggan/smithsonian_butterflies_subset")

for idx, example in enumerate(ds["train"]):
    img = example["image"]
    img_path = os.path.join(OUT_DIR, f"{idx:06d}.jpg")
    img.save(img_path)
    if idx >= 5000:  # limit if you only want around this many images
        break

print("Saved images to", OUT_DIR)
print("Count:", len(os.listdir(OUT_DIR)))


Repo card metadata block was not found. Setting CardData to empty.


Saved images to ./smithsonian-butterflies-subset/images
Count: 1000


In [6]:
dataset = ImageFolderDataset(CONFIG["data_dir"], image_size=CONFIG["image_size"], channels=CONFIG["channels"])
print(f"Found {len(dataset)} images")


dataloader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"], drop_last=True)

Found 1000 images


# 🔧 Time Embeddings & Residual Blocks

This section defines two important components used inside the U-Net for DDPM:  
1) a sinusoidal timestep embedding, and  
2) a residual block with optional time conditioning.

---

## 🔹 SinusoidalPosEmb

This module converts a scalar timestep **t** into a high-dimensional vector using the classic Transformer-style sinusoidal embedding.

### How it works:
- Takes an input tensor `x` containing timesteps `[B]` or `[B, 1]`.
- Creates a set of exponentially-scaled frequencies.
- Produces a vector of `sin` and `cos` values at those frequencies.
- Output shape: `[B, dim]`.

This embedding gives the model a continuous way to understand the diffusion timestep.

---

## 🔹 ResidualBlock

A standard ResNet-style block adapted for diffusion models, with optional time conditioning.

### Key components:
- Two Conv2D layers (3×3) with GroupNorm and SiLU activation.
- A shortcut connection (1×1 conv) if channel sizes change.
- Optional **time_mlp**:  
  - Takes a time embedding and maps it to `out_ch`  
  - Added to feature maps after the first convolution

### Forward pass:
1. Normalize + SiLU + Conv  
2. Add time embedding (if provided)  
3. Normalize + SiLU + Conv  
4. Add the shortcut connection  

This block allows the U-Net to incorporate timestep information into its feature maps.



In [7]:
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        # x: (B,) or (B,1)
        device = x.device
        half = self.dim // 2
        emb = math.log(10000) / (half - 1)
        emb = torch.exp(torch.arange(half, device=device) * -emb)
        emb = x[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb

class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch, time_emb_dim=None):
        super().__init__()
        self.norm1 = nn.GroupNorm(8, in_ch)
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.norm2 = nn.GroupNorm(8, out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        if in_ch != out_ch:
            self.nin_shortcut = nn.Conv2d(in_ch, out_ch, 1)
        else:
            self.nin_shortcut = nn.Identity()
        if time_emb_dim is not None:
            self.time_mlp = nn.Sequential(nn.SiLU(), nn.Linear(time_emb_dim, out_ch))
        else:
            self.time_mlp = None

    def forward(self, x, t_emb=None):
        h = self.conv1(F.silu(self.norm1(x)))
        if self.time_mlp is not None and t_emb is not None:
            h = h + self.time_mlp(t_emb)[:,:,None,None]
        h = self.conv2(F.silu(self.norm2(h)))
        return h + self.nin_shortcut(x)


# 🧠 UNet Architecture for DDPM (Noise Prediction)

This UNet is designed for Denoising Diffusion Probabilistic Models (DDPM).  
It takes a noisy image **xₜ** and a timestep **t**, and predicts the added noise **ε**.

The model has:
- Time embedding MLP  
- Downsampling path  
- Bottleneck  
- Upsampling path  
- Skip connections  
- Final reconstruction block  

---

## 🔹 Time Embedding MLP
We embed the timestep `t` using a sinusoidal positional encoding, then pass it through an MLP:
- Helps the model understand *when* in the diffusion process an image is.
- Output dimension: `time_emb_dim`.

---

## 🔹 Initial Convolution
A simple 3×3 conv expands the input image channels into the base feature dimension.

---

## 🔹 Downsampling Blocks
Each downsampling stage contains:
1. Two `ResidualBlock`s (with time conditioning)
2. A stride-2 convolution for deterministic downsampling

At each stage:
- Channels increase based on `ch_mults`
- Feature maps are stored as *skip connections*

---

## 🔹 Bottleneck
Two residual blocks operating at the lowest spatial resolution:
- Provides nonlinear mixing of global context

---

## 🔹 Upsampling Blocks
Each upsampling stage consists of:
1. Nearest-neighbor upsample (keeps spatial alignment clean)
2. Concatenate with matching skip connection
3. Two `ResidualBlock`s

If any shape mismatch occurs (rare), a safe `_match_spatial` function fixes it by center-crop or padding.

---

## 🔹 Final Output Layer
GroupNorm → SiLU → 3×3 conv  
Outputs a tensor matching the input image channels, representing **predicted noise ε**.

---

## 🔹 Forward Pass Summary
1. Compute time embedding  
2. Initial convolution  
3. Down path → save skip connections  
4. Bottleneck  
5. Up path → merge skips + upsample  
6. Final conv → return noise prediction

This architecture matches common DDPM/UNet designs used in diffusion models.



In [8]:
class UNet(nn.Module):
    """
    UNet that predicts noise epsilon for DDPM.
    Uses conv stride-2 for downsampling (deterministic),
    nearest upsampling for upsampling (keeps integers).
    """
    def __init__(self, in_channels=3, base_ch=64, ch_mults=(1,2,2,4), time_emb_dim=256):
        super().__init__()
        self.time_mlp = nn.Sequential(
            SinusoidalPosEmb(time_emb_dim),
            nn.Linear(time_emb_dim, time_emb_dim),
            nn.SiLU(),
            nn.Linear(time_emb_dim, time_emb_dim),
        )

        # initial conv
        self.init_conv = nn.Conv2d(in_channels, base_ch, 3, padding=1)

        # down blocks: for each level, two residual blocks then a conv stride-2 downsample
        in_ch = base_ch
        self.downs = nn.ModuleList()
        for mult in ch_mults:
            out_ch = base_ch * mult
            block = nn.ModuleDict({
                "res1": ResidualBlock(in_ch, out_ch, time_emb_dim),
                "res2": ResidualBlock(out_ch, out_ch, time_emb_dim),
                # deterministic downsample conv
                "down": nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=2, padding=1)
            })
            self.downs.append(block)
            in_ch = out_ch

        # bottleneck
        self.mid1 = ResidualBlock(in_ch, in_ch, time_emb_dim)
        self.mid2 = ResidualBlock(in_ch, in_ch, time_emb_dim)

        # up blocks: nearest upsample, concat skip, then two residuals
        self.ups = nn.ModuleList()
        rev_ch_mults = list(reversed(ch_mults))
        for i, mult in enumerate(rev_ch_mults):
            out_ch = base_ch * mult
            block = nn.ModuleDict({
                "up": nn.Upsample(scale_factor=2, mode='nearest'),
                # after concat channels will be in_ch (current) + out_ch (skip)
                "res1": ResidualBlock(in_ch + out_ch, out_ch, time_emb_dim),
                "res2": ResidualBlock(out_ch, out_ch, time_emb_dim),
            })
            self.ups.append(block)
            in_ch = out_ch

        # final layers
        self.final_block = nn.Sequential(
            nn.GroupNorm(8, in_ch),
            nn.SiLU(),
            nn.Conv2d(in_ch, in_channels, 3, padding=1)
        )

    def forward(self, x, t):
        """
        x: (B, C, H, W)
        t: (B,) long or float tensor of timesteps
        """
        t_emb = self.time_mlp(t.float())

        h = self.init_conv(x)
        skips = []

        # down
        for block in self.downs:
            h = block["res1"](h, t_emb)
            h = block["res2"](h, t_emb)
            skips.append(h)               # store this BEFORE downsample
            h = block["down"](h)         # size //= 2

        # bottleneck
        h = self.mid1(h, t_emb)
        h = self.mid2(h, t_emb)

        # up
        # iterate over ups and corresponding skip in reversed order
        for block, skip in zip(self.ups, reversed(skips)):
            h = block["up"](h)   # nearest upsample -> should exactly match skip H/W
            # if there's any mismatch due to rounding, center-crop/pad to match (safe fallback)
            if h.shape[-2:] != skip.shape[-2:]:
                # center-crop or pad h to skip's spatial size (this should rarely be used)
                h = self._match_spatial(h, skip)
            h = torch.cat([h, skip], dim=1)
            h = block["res1"](h, t_emb)
            h = block["res2"](h, t_emb)

        return self.final_block(h)

    @staticmethod
    def _match_spatial(a, b):
        """
        Make tensor `a` match spatial size of `b` by either cropping or padding.
        a, b: tensors with shape (B, C, H, W)
        Returns modified a with spatial size equal to b.
        """
        ah, aw = a.shape[-2:]
        bh, bw = b.shape[-2:]
        # If a is larger -> crop center
        if ah > bh or aw > bw:
            top = (ah - bh) // 2
            left = (aw - bw) // 2
            a = a[..., top:top+bh, left:left+bw]
            return a
        # If a is smaller -> pad equally
        pad_h = bh - ah
        pad_w = bw - aw
        pad = (pad_w//2, pad_w - pad_w//2, pad_h//2, pad_h - pad_h//2)  # (left,right,top,bottom)
        a = F.pad(a, pad)
        return a


# ⚡ DDPM Class — Forward & Reverse Diffusion

This class implements the core math of a **Denoising Diffusion Probabilistic Model (DDPM)**:
- Forward process: gradually adds noise  
- Reverse process: removes noise using the UNet  
- Sampling loop: generates images from pure noise  

## ✔️ Summary
This DDPM class handles:
- Forward noising  
- Reverse denoising  
- Full sampling loop  
- Integration with the UNet noise predictor  

Together, this forms the mathematical backbone of diffusion models.

In [9]:
class DDPM:
    def __init__(self, timesteps=1000, beta_start=1e-4, beta_end=0.02, device='cpu'):
        self.timesteps = timesteps
        self.device = device
        self.beta = torch.linspace(beta_start, beta_end, timesteps, device=device)
        self.alpha = 1. - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)

    def q_sample(self, x_start, t, noise=None):
        # x_start: [B,C,H,W], t: [B]
        if noise is None:
            noise = torch.randn_like(x_start)
        sqrt_alpha_bar = torch.sqrt(self.alpha_bar[t]).reshape(-1, 1, 1, 1)
        sqrt_one_minus_alpha_bar = torch.sqrt(1. - self.alpha_bar[t]).reshape(-1, 1, 1, 1)
        return sqrt_alpha_bar * x_start + sqrt_one_minus_alpha_bar * noise, noise

    @torch.no_grad()
    def p_sample(self, model, x_t, t):
        # one reverse diffusion step from t -> t-1
        betat = self.beta[t].reshape(-1,1,1,1)
        alpha_t = self.alpha[t].reshape(-1,1,1,1)
        alpha_bar_t = self.alpha_bar[t].reshape(-1,1,1,1)

        # predict noise
        eps_theta = model(x_t, t)

        coef1 = 1./torch.sqrt(alpha_t)
        coef2 = (1 - alpha_t) / torch.sqrt(1 - alpha_bar_t)
        mean = coef1 * (x_t - coef2 * eps_theta)

        # no noise when t == 0
        noise = torch.randn_like(x_t) if (t > 0).any() else torch.zeros_like(x_t)
        sigma_t = torch.sqrt(betat)
        # when t==0, sigma_t should be 0; but our indexing below passes t>=0
        # return sample
        return mean + sigma_t * noise

    @torch.no_grad()
    def sample(self, model, shape, device, progress=True):
        model.eval()
        x = torch.randn(shape, device=device)
        for i in tqdm(reversed(range(self.timesteps)), desc='sampling', total=self.timesteps, disable=not progress):
            t = torch.full((shape[0],), i, dtype=torch.long, device=device)
            x = self.p_sample(model, x, t)
        model.train()
        return x

# Section - DDPM Training Loop Explanation

This section performs the full training of the U-Net + DDPM model.  
It includes model initialization, optimizer setup, timestep sampling, noise prediction, and model checkpoint saving.

---

## 1. Sigmoid Helper
A simple helper function:
- Computes the sigmoid value using the standard formula `1 / (1 + exp(-x))`.

Used only when needed for additional normalization or scaling.

---

## 2. Initializing the Model and Optimizer

- `UNet` is the neural network that predicts noise.  
- `DDPM` provides the forward and reverse diffusion equations.  
- `Adam` optimizer updates the U-Net parameters.  
- `MSELoss` measures how close predicted noise is to true noise.

All components are moved to the selected device (CPU or GPU).

---

## 3. Timestep Embedding Helper

The model expects each timestep `t` as a float tensor so it can be encoded by the sinusoidal time embedding.



In [10]:
def sigmoid(x):
    return 1/(1+math.exp(-x))

# init model, ddpm helper, optimizer
device = CONFIG['device']
model = UNet(in_channels=CONFIG['channels']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG['lr'])
criterion = nn.MSELoss()

ddpm = DDPM(timesteps=CONFIG['timesteps'], beta_start=CONFIG['beta_start'], beta_end=CONFIG['beta_end'], device=device)

# small helper to convert integer timesteps to embeddings tensor for model
# Note: model expects t as float tensor used in SinusoidalPosEmb

def make_timestep_tensor(t_batch, device=device):
    # t_batch: tensor of longs
    return t_batch.float().to(device)

loss_history = []
step = 0
start_time = datetime.now()

for epoch in range(1, CONFIG['epochs'] + 1):
    pbar = tqdm(dataloader, desc=f"Epoch {epoch}/{CONFIG['epochs']}")
    running_loss = 0.0
    for batch in pbar:
        batch = batch.to(device)
        bsz = batch.size(0)

        # sample random timesteps for each image in the batch
        t = torch.randint(0, ddpm.timesteps, (bsz,), device=device)

        x_t, noise = ddpm.q_sample(batch, t)

        # model predicts noise
        pred = model(x_t, make_timestep_tensor(t))
        loss = criterion(pred, noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        running_loss += loss.item()
        step += 1

        if step % 50 == 0:
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch {epoch} finished — avg loss: {epoch_loss:.6f}")

    # save checkpoint
    if epoch % CONFIG['save_every'] == 0 or epoch == CONFIG['epochs']:
        ckpt = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'loss_history': loss_history,
        }
        torch.save(ckpt, os.path.join(CONFIG['save_dir'], f'ddpm_epoch_{epoch}.pt'))
        print(f"Saved checkpoint: ddpm_epoch_{epoch}.pt")

# save final model
torch.save({'model_state_dict': model.state_dict(), 'loss_history': loss_history}, os.path.join(CONFIG['save_dir'], 'ddpm_final.pt'))
print('Training finished — elapsed:', datetime.now() - start_time)


Epoch 1/200:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 1 finished — avg loss: 0.193717


Epoch 2/200:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 2 finished — avg loss: 0.062328


Epoch 3/200:   0%|          | 0/125 [00:00<?, ?it/s]

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2ba278c280>
Traceback (most recent call last):
  File "/venv/main/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/venv/main/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
  File "/venv/main/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    Exception ignored in: assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError<function _MultiProcessingDataLoaderIter.__del__ at 0x7f2ba278c280>: 
Traceback (most recent call last):
can only test a child process
  File "/venv/main/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f2ba278c280>
Traceback (most recent call last):
  File "/venv/main/l

Epoch 131 finished — avg loss: 0.016169


Epoch 132/200:   0%|          | 0/125 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: IOStream.flush timed out
IOStream.flush timed out
Exception ignored in: Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2ba278c280><function _MultiProcessingDataLoaderIter.__del__ at 0x7f2ba278c280><function _MultiProcessingDataLoaderIter.__del__ at 0x7f2ba278c280>
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f2ba278c280><function _MultiProcessingDataLoaderIter.__del__ at 0x7f2ba278c280>

Traceback (most recent call last):


Traceback (most recent call last):
  File "/venv/main/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
Traceback (most recent call last):
Traceback (most recent call last):
  File "/venv/main/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
Traceback (most recent call last):
  File "/venv/main/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
      File "/ven

KeyboardInterrupt: 

# Section - Plotting Loss and Generating DDPM Samples

This part of the notebook visualizes the training progress and uses the trained DDPM model to generate sample images.

---

## 1. Plotting Training Loss

The first block plots the loss history collected during training.

- `loss_history` contains the MSE noise prediction loss for every training step.
- The curve helps verify:
  - Whether the model is converging
  - If the learning rate is appropriate
  - If training is stable over time

A simple Matplotlib line plot is used:
- X-axis: training steps
- Y-axis: MSE loss value

The grid and tight layout improve readability.

---

## 2. Generating Images with DDPM

After training is complete, the model is switched to evaluation mode using `model.eval()`.  
Image generation proceeds as follows:

1. Generate a batch of pure noise images.
2. DDPM runs the reverse diffusion process, step-by-step, removing noise.
3. The output is a batch of generated images.

Key details:
- The code generates about 50 images.
- Sampling is done in mini-batches (`bs = 5`) to save GPU memory.
- Outputs are in the range [-1, 1], so they are rescaled to [0, 1].

---

## 3. Displaying Generated Samples

The generated images are arranged in a grid:

- 10 columns
- Enough rows to cover all samples
- `imshow` displays each image after permuting channel order
- Any empty grid cells are hidden

The grid provides a quick overview of sample quality.

---

## 4. Saving Generated Images

Each output is saved individually as a `.png` file:

- Images are scaled to 0-255
- Converted to `uint8`
- Saved in a directory inside `save_dir` under `samples`

This allows:
- Reviewing image quality
- Using samples for datasets, demos, experiments
- Making GIFs or videos from sample sequences

---

## Summary

This section completes the DDPM workflow:
- It visualizes convergence with a loss plot.
- It uses the trained model to generate new images.
- It displays the generated images in a grid.
- It saves all generated samples to disk for later use.


In [ ]:
torch.save({'model_state_dict': model.state_dict(), 'loss_history': loss_history}, os.path.join('./', 'ddpm_final.pt'))

In [ ]:
!ls
!pwd

In [ ]:
# plt.figure(figsize=(8,4))
# plt.plot(np.array(loss_history))
# plt.title('U-Net training loss (MSE)')
# plt.xlabel('Training steps')
# plt.ylabel('MSE Loss')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# %%
# 8) Generate ~200 images and plot them
num_samples = 150
bs = 5
samples = []
model.eval()
with torch.no_grad():
    for i in range(math.ceil(num_samples/bs)):
        cur = min(bs, num_samples - i*bs)
        imgs = ddpm.sample(model, (cur, CONFIG['channels'], CONFIG['image_size'], CONFIG['image_size']), device)
        # denormalize (model outputs in same normalized [-1,1] range as inputs)
        imgs = imgs.clamp(-1,1)
        imgs = (imgs + 1) / 2
        imgs = imgs.cpu()
        samples.append(imgs)

samples = torch.cat(samples, dim=0)[:num_samples]

# plot grid
cols = 10
rows = math.ceil(num_samples/cols)
# fig, axes = plt.subplots(rows, cols, figsize=(cols*1.8, rows*1.8))
# for i in range(rows*cols):
#     r = i // cols
#     c = i % cols
#     ax = axes[r, c] if rows>1 else axes[c]
#     ax.axis('off')
#     if i < num_samples:
#         img = samples[i].permute(1,2,0).numpy()
#         ax.imshow(img)
#     else:
#         ax.set_visible(False)
# plt.suptitle('DDPM samples')
# plt.tight_layout()
# plt.show()

# Save generated images individually
out_samples_dir = os.path.join(CONFIG['save_dir'], 'samples')
os.makedirs(out_samples_dir, exist_ok=True)
for i in range(num_samples):
    img = (samples[i].permute(1,2,0).numpy() * 255).astype(np.uint8)
    Image.fromarray(img).save(os.path.join(out_samples_dir, f'sample_{i:03d}.png'))
print(f"Saved {num_samples} samples to {out_samples_dir}")

sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

...

sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
!zip -r samples_300.zip ddpm_checkpoints/samples/

  adding: ddpm_checkpoints/samples/ (stored 0%)
  adding: ddpm_checkpoints/samples/sample_000.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_001.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_002.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_003.png (deflated 0%)
  adding: ddpm_checkpoints/samples/sample_004.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_005.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_006.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_007.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_008.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_009.png (deflated 1%)
  adding: ddpm_checkpoints/samples/sample_010.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_011.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_012.png (stored 0%)
  adding: ddpm_checkpoints/samples/sample_013.png (deflated 0%)
  adding: ddpm_checkpoints/samples/sample_014.png (stored 0%)
  adding: ddpm_c

In [ ]:
import gdown
url = "./ddpm_checkpoints"
gdown.download_folder(url, quiet=True)


In [ ]:
!cd ddpm_checkpoints 

In [ ]:
!zip -r ddpm_checkpoints.zip .

In [ ]:
!pwd